In [29]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import requests
from newspaper import Article
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/dylan/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [45]:
trump_dict = {'name': 'donald trump', 'age': '75 years old', 'occupation': 'president'}
najib_dict = {'name': 'najib razak', 'age': '68 years old', 'occupation': 'prime minister'}

In [38]:
new_words = {
    'crushes': 10,
    'beats': 5,
    'misses': -5,
    'trouble': -10,
    'falls': -100,
    'bankrupt': -100,
    'fraud': -100,
    'crime': -100,
    'prison': -100,
    'embezzlement': -100,
    'money_trail':-100,
    'jail': -100,
    'sentenced':-100,
    'forfeiture': -100,
    'ill-gotten gain':-100,
    'laundering':-100
}


vader = SentimentIntensityAnalyzer()

vader.lexicon.update(new_words)

In [39]:
def generate_link(identifier_dict, no_of_articles):
    link_start = "https://www.google.com/search?q="
    link_end = "&sxsrf=ALeKk01K1bOuJFHjy4HBARo1cRpUYakYPg:1629640327633&source=lnms&tbm=nws&sa=X&sqi=2&ved=2ahUKEwiu29um48TyAhWGqpUCHYuoAlcQ_AUoAnoECAEQBA&biw=1441&bih=718&dpr=2" 
    link_query = ""
    for identifier in identifier_dict.values():        
        link_query += identifier.replace(' ','+') + '+'
    return link_start + link_query + link_end + "&num=" + str(no_of_articles)

def article_extraction(link):
    article = Article(link)
    article.download()
    try:
        article.parse()
    except:
        pass
    return article.text

def sentiment_analysis(text):
    return vader.polarity_scores(text)


In [4]:
testlink = generate_link(trump_dict, 30)
print(testlink)

https://www.google.com/search?q=donald+trump+75+years+old+president+&sxsrf=ALeKk01K1bOuJFHjy4HBARo1cRpUYakYPg:1629640327633&source=lnms&tbm=nws&sa=X&sqi=2&ved=2ahUKEwiu29um48TyAhWGqpUCHYuoAlcQ_AUoAnoECAEQBA&biw=1441&bih=718&dpr=2&num=30


In [40]:
# Main Function
def search_articles_on_individual(individual_dict, no_of_articles):
    def generate_link(identifier_dict, no_of_articles):
        link_start = "https://www.google.com/search?q="
        link_end = "&sxsrf=ALeKk01K1bOuJFHjy4HBARo1cRpUYakYPg:1629640327633&source=lnms&tbm=nws&sa=X&sqi=2&ved=2ahUKEwiu29um48TyAhWGqpUCHYuoAlcQ_AUoAnoECAEQBA&biw=1441&bih=718&dpr=2" 
        link_query = ""
        for identifier in identifier_dict.values():        
            link_query += identifier.replace(' ','+') + '+'
        return link_start + link_query + link_end + "&num=" + str(no_of_articles)
    
    link = generate_link(individual_dict, no_of_articles)
    req = Request(link, headers = {'User-Agent': 'Mozilla/5.0'})

    webpage = urlopen(req).read()

    links = []
    with requests.Session() as c:
        soup = BeautifulSoup(webpage, 'html5lib')
        #print(soup)
        for item in soup.find_all('div', attrs = {'class': "ZINbbc xpd O9g5cc uUPGi"}):
            current_dict = {}
            raw_link = (item.find('a', href = True)['href'])
            link = (raw_link.split("/url?q=")[1]).split('&sa=U&')[0]
            title = (item.find('div',attrs = {'class': 'BNeawe vvjwJb AP7Wnd'})).get_text()
            description  = (item.find('div',attrs = {'class': 'BNeawe s3v9rd AP7Wnd'}).get_text())
            time = description.split(" · ")[0]
            #print(description)
            descript = description.split(" · ")[1]
            current_dict['title'] = title
            current_dict['time'] = time
            current_dict['description'] = descript
            current_dict['link'] = link
            current_dict['text'] = article_extraction(link)
            current_dict['sentiment'] = sentiment_analysis(article_extraction(link))
            links.append(current_dict)
    return links

In [48]:
#trump_query = search_articles_on_individual(trump_dict, 30)
najib_query = search_articles_on_individual(najib_dict, 30)

In [49]:
najib_query[0]

{'title': <div class="BNeawe vvjwJb AP7Wnd">High Court judge furious at Najib's absence from 1MDB trial</div>,
 'time': '5 days ago',
 'description': 'KUALA LUMPUR (THE STAR/ASIA NEWS NETWORK) - High Court judge Collin Lawrence Sequerah rebuked former prime minister Najib Razak,...',
 'link': 'https://www.straitstimes.com/asia/se-asia/high-court-judge-furious-at-najibs-absence-from-1mdb-trial',
 'text': 'KUALA LUMPUR (THE STAR/ASIA NEWS NETWORK) - High Court judge Collin Lawrence Sequerah rebuked former prime minister Najib Razak, as he was absent in court for his 1Malaysia Development Bhd (1MDB) trial on Thursday (Aug 19) morning.\n\nNajib\'s lawyer Rahmat Hazlan informed the court that the Pekan MP, together with 114 other MPs, had been summoned to the palace to confirm their support for Umno vice-president Ismail Sabri Yaakob as the new prime minister.\n\n"The King wants to interview the MPs one by one. My client is scheduled to leave the World Trade Centre (WTCKL) at 10.30am. I bel

In [50]:
df = pd.DataFrame(najib_query)
df

,title,time,description,link,text,sentiment
0,[High Court judge furious at Najib's absence f...,5 days ago,KUALA LUMPUR (THE STAR/ASIA NEWS NETWORK) - Hi...,https://www.straitstimes.com/asia/se-asia/high...,KUALA LUMPUR (THE STAR/ASIA NEWS NETWORK) - Hi...,"{'neg': 0.26, 'neu': 0.691, 'pos': 0.049, 'com..."
1,"[Najib Razak ditches court for king's meeting,...",5 days ago,Former Malaysian prime minister Najib Razak ch...,https://coconuts.co/kl/news/najib-razak-ditche...,Former Malaysian prime minister Najib Razak ch...,"{'neg': 0.062, 'neu': 0.888, 'pos': 0.05, 'com..."
2,"[Judge unhappy Najib absent, orders him to tur...",5 days ago,... Najib Tun Razak's (pix) 1Malaysia Developm...,https://www.thesundaily.my/local/judge-unhappy...,KUALA LUMPUR: The judge presiding over Datuk S...,"{'neg': 0.248, 'neu': 0.696, 'pos': 0.056, 'co..."
3,[KL judge furious over Najib's absence at 1MDB...,5 days ago,KUALA LUMPUR • Kuala Lumpur High Court judge C...,https://www.straitstimes.com/asia/se-asia/kl-j...,KUALA LUMPUR • Kuala Lumpur High Court judge C...,"{'neg': 0.291, 'neu': 0.665, 'pos': 0.044, 'co..."
4,[Here Are the Contenders to Become Malaysia's ...,1 week ago,"Najib Razak, 68. The former prime minister, wh...",http://www.bnnbloomberg.ca/here-are-the-conten...,(Bloomberg) -- The race to become Malaysia’s n...,"{'neg': 0.152, 'neu': 0.769, 'pos': 0.079, 'co..."
5,[Najib's lawyer accuses former 1MDB CEO of CBT...,1 day ago,Lead defence counsel Tan Sri Muhammad Shafee A...,https://www.thestar.com.my/news/nation/2021/08...,KUALA LUMPUR: Former 1Malaysia Development Bhd...,"{'neg': 0.508, 'neu': 0.442, 'pos': 0.051, 'co..."
6,[Najib gets break from 1MDB trial to attend po...,1 week ago,"He faces 20 years' jail and fine, if convicted...",https://www.nst.com.my/news/nation/2021/08/718...,,"{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound..."
7,[1MDB paid Bloomberg to list fund and boost it...,5 days ago,Former 1MDB CEO Mohd Hazem Abd Rahman agreed t...,https://www.thestar.com.my/news/nation/2021/08...,KUALA LUMPUR: 1Malaysia Development Bhd (1MDB)...,"{'neg': 0.202, 'neu': 0.727, 'pos': 0.071, 'co..."
8,"[High Court asks Najib, Azhar, Rashidi to file...",3 weeks ago,... today instructed former prime minister Dat...,https://www.theedgemarkets.com/article/high-co...,KUALA LUMPUR (Aug 3): The High Court has today...,"{'neg': 0.27, 'neu': 0.698, 'pos': 0.033, 'com..."
9,"[Najib should not be in the dock but Hazem, sa...",6 days ago,KUALA LUMPUR: Datuk Seri Najib Razak's lawyer ...,https://www.nst.com.my/news/crime-courts/2021/...,,"{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound..."
